# Approximate q-learning

In this notebook you will teach a lasagne neural network to do Q-learning.

__Frameworks__ - we'll accept this homework in any deep learning framework. For example, it translates to TensorFlow almost line-to-line. However, we recommend you to stick to theano/lasagne unless you're certain about your skills in the framework of your choice.

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [3]:
import gym
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
env = gym.make("CartPole-v0")
env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

#plt.imshow(env.render("rgb_array"))

[2017-04-21 09:09:26,153] Making new env: CartPole-v0


In [127]:
batch_size = 4
L1_size = 50

gamma = 0.99

In [6]:
print("n_actions={}, state_dim={}".format(n_actions, state_dim))

n_actions=2, state_dim=(4,)


# Approximate (deep) Q-learning: building the network

In this section we will build and train naive Q-learning with theano/lasagne

First step is initializing input variables

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc_1 = nn.Linear(int(np.prod(state_dim)), L1_size)
        self.fc_out = nn.Linear(L1_size, n_actions)
        
    def forward(self, x):
        x = F.relu(self.fc_1(x))
        x = self.fc_out(x)
        return x

In [46]:
model = Net()
print(model)

Net (
  (fc_1): Linear (4 -> 50)
  (fc_out): Linear (50 -> 2)
)


In [47]:
s = env.reset()

In [146]:
def states_to_var(s):
    return Variable(torch.from_numpy(np.array(s, dtype=np.float32)), volatile=False)

In [140]:
s_var = states_to_var([s])
print(s_var)

Variable containing:
1.00000e-02 *
 -4.0394 -4.7849 -2.2721  3.9101
[torch.FloatTensor of size 1x4]



In [59]:
q = model(s_var)

In [77]:
print(q)
q.max(dim=1)[1]

Variable containing:
 0.0902  0.1355
[torch.FloatTensor of size 1x2]



Variable containing:
 1
[torch.IntTensor of size 1x1]

In [58]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# import theano
# import theano.tensor as T

# #create input variables. We'll support multiple states at once


# current_states = T.matrix("states[batch,units]")
# actions = T.ivector("action_ids[batch]")
# rewards = T.vector("rewards[batch]")
# next_states = T.matrix("next states[batch,units]")
# is_end = T.ivector("vector[batch] where 1 means that session just ended")

In [ ]:
# import lasagne
# from lasagne.layers import *

# #input layer
# l_states = InputLayer((None,)+state_dim)


# <Your architecture. Please start with a single-layer network>


# #output layer
# l_qvalues = DenseLayer(<previous_layer>,num_units=n_actions,nonlinearity=None)

#### Predicting Q-values for `current_states`

In [ ]:
#get q-values for ALL actions in current_states
predicted_qvalues = get_output(l_qvalues,{l_states:current_states})

In [ ]:
#compiling agent's "GetQValues" function
get_qvalues = <compile a function that takes current_states and returns predicted_qvalues>

In [ ]:
#select q-values for chosen actions
predicted_qvalues_for_actions = predicted_qvalues[T.arange(actions.shape[0]),actions]

#### Loss function and `update`
Here we write a function similar to `agent.update`.

In [ ]:
#predict q-values for next states
predicted_next_qvalues = get_output(l_qvalues,{l_states:<theano input with for states>})


#Computing target q-values under 
gamma = 0.99
target_qvalues_for_actions = <target Q-values using rewards and predicted_next_qvalues>

#zero-out q-values at the end
target_qvalues_for_actions = (1-is_end)*target_qvalues_for_actions

#don't compute gradient over target q-values (consider constant)
target_qvalues_for_actions = theano.gradient.disconnected_grad(target_qvalues_for_actions)

In [ ]:

#mean squared error loss function
loss = <mean squared between target_qvalues_for_actions and predicted_qvalues_for_actions>


In [ ]:
#all network weights
all_weights = get_all_params(l_qvalues,trainable=True)

#network updates. Note the small learning rate (for stability)
updates = lasagne.updates.sgd(loss,all_weights,learning_rate=1e-4)

In [ ]:
#Training function that resembles agent.update(state,action,reward,next_state) 
#with 1 more argument meaning is_end
train_step = theano.function([current_states,actions,rewards,next_states,is_end],
                             updates=updates)

### Playing the game

In [147]:
def choose_action(q_values, epsilon):
    if np.random.rand() < epsilon:
        return torch.LongTensor([np.random.choice(n_actions)])
    else:
        return q_values.max(dim=1)[1].data[0]

In [167]:
optimizer.zero_grad()

In [168]:
s = env.reset()
q = model(states_to_var([s]))
a = choose_action(q, epsilon=0.25).cpu()[0]

new_s, r, done, _ = env.step(a)
new_q = model(states_to_var([new_s]))

chosen_q = q[:, a]
valid_q = r + gamma * new_q.max(dim=1)[0]

In [169]:
loss = nn.MSELoss(size_average=False)

In [170]:
l = (valid_q - chosen_q)**2
print(l)

Variable containing:
 0.9947
[torch.FloatTensor of size 1x1]



In [171]:
l.backward()

In [122]:
q[:,a]

Variable containing:
 0.1084
[torch.FloatTensor of size 1]

In [ ]:
epsilon = 0.25 #initial epsilon

def generate_session(t_max=1000):
    """play env with approximate q-learning agent and train it at the same time"""
    
    total_reward = 0
    s = env.reset()
    
    for t in range(t_max):    
        model(states)
        a = choose_action(s, epsilon)
        new_s,r,done,info = env.step(a)
        
        optimizer.zero_grad()
        
        
        
        #train agent one step. Note that we use one-element arrays instead of scalars 
        #because that's what function accepts.
        train_step([s],[a],[r],[new_s],[done])
        
        total_reward+=r
        
        s = new_s
        if done: break
            
    return total_reward
        

In [ ]:
for i in range(100):
    
    rewards = [generate_session() for _ in range(100)] #generate new sessions
    
    epsilon*=0.95
    
    print ("mean reward:%.3f\tepsilon:%.5f"%(np.mean(rewards),epsilon))

    if np.mean(rewards) > 300:
        print ("You Win!")
        break
        
    assert epsilon!=0, "Please explore environment"

### Video

In [ ]:
epsilon=0 #Don't forget to reset epsilon back to initial value if you want to go on training

In [ ]:
#record sessions
import gym.wrappers
env = gym.wrappers.Monitor(env,directory="videos",force=True)
sessions = [generate_session() for _ in range(100)]
env.close()
#unwrap 
env = env.env.env
#upload to gym
#gym.upload("./videos/",api_key="<your_api_key>") #you'll need me later

#Warning! If you keep seeing error that reads something like"DoubleWrapError",
#run env=gym.make("CartPole-v0");env.reset();

In [ ]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices